# Real-Time Network Emulation Visualization

This notebook provides real-time visualization of the running SiNE emulation.

## Prerequisites

1. **Channel server must be running**: `uv run sine channel-server`
2. **Emulation must be deployed**: `sudo $(which uv) run sine deploy <topology.yaml>`

## How It Works

- Polls the channel server's `/api/visualization/state` endpoint
- Displays cached path data from previous channel computations
- Shows wireless channel metrics (delay spread, K-factor, coherence bandwidth)
- Updates every 1 second (configurable)

**Important**: No computation is performed in this notebook - all data is pre-computed and cached by the channel server during emulation deployment/updates.

In [ ]:
# Cell 1: Setup and Configuration
import asyncio
import httpx
from IPython.display import clear_output, display
from typing import Any

# Configuration
CHANNEL_API = "http://localhost:8000"
UPDATE_INTERVAL_SEC = 1.0  # Poll interval
MAX_RENDER_PATHS = 5       # Limit paths per link for performance

In [ ]:
# Cell 2: Helper Functions
async def fetch_visualization_state() -> dict[str, Any]:
    """
    Fetch complete visualization state from channel server.

    Returns scene geometry, device positions, and CACHED paths.
    No computation required - instant response.
    """
    async with httpx.AsyncClient(timeout=5.0) as client:
        response = await client.get(f"{CHANNEL_API}/api/visualization/state")
        response.raise_for_status()
        return response.json()

In [ ]:
# Cell 3: Rendering Functions (ENHANCED with Wireless Metrics)
def display_visualization(viz_state: dict[str, Any]) -> None:
    """
    Display visualization state with wireless channel analysis.
    
    Args:
        viz_state: Visualization state from channel server
    """
    print(f"=== Visualization State (Cache: {viz_state['cache_size']} links) ===")
    print(f"Scene: {viz_state.get('scene_file', 'N/A')}\n")

    # Display device positions
    print("Devices:")
    for device in viz_state["devices"]:
        pos = device["position"]
        print(f"  {device['name']}: ({pos['x']:.1f}, {pos['y']:.1f}, {pos['z']:.1f})")

    # Display detailed channel information
    print(f"\n{'='*70}")
    print("WIRELESS CHANNEL ANALYSIS")
    print(f"{'='*70}")

    for link_data in viz_state["paths"]:
        print(f"\nLink: {link_data['tx_name']} → {link_data['rx_name']}")
        print(f"{'-'*70}")

        # Basic link info
        print(f"Distance: {link_data['distance_m']:.1f} m")
        print(f"Paths: {link_data['num_paths_shown']}/{link_data['num_paths_total']} "
              f"({link_data.get('power_coverage_percent', 100):.1f}% power)")

        # Delay spread analysis (ISI characterization)
        rms_ds_ns = link_data.get('rms_delay_spread_ns', 0)
        bc_mhz = link_data.get('coherence_bandwidth_hz', 0) / 1e6

        print(f"\nDelay Characteristics:")
        print(f"  RMS Delay Spread (τ_rms): {rms_ds_ns:.2f} ns")
        print(f"  Coherence Bandwidth (Bc): {bc_mhz:.1f} MHz")

        # Frequency selectivity assessment
        # Assume 80 MHz signal BW (adjust based on your config)
        signal_bw_mhz = 80  # TODO: Get from link config
        if bc_mhz > signal_bw_mhz:
            print(f"  ✓ Frequency-flat channel (Bc > BW)")
        else:
            print(f"  ⚠ Frequency-selective channel (Bc ≈ BW)")
            print(f"    ISI may be significant - OFDM recommended")

        # LOS/NLOS classification via Rician K-factor
        k_factor = link_data.get('k_factor_db')
        dominant_type = link_data.get('dominant_path_type', 'unknown')

        print(f"\nChannel Classification:")
        if k_factor is not None:
            print(f"  Rician K-factor: {k_factor:.1f} dB")
            if k_factor > 10:
                print(f"  → Strong LOS component (K > 10 dB)")
            elif k_factor > 0:
                print(f"  → Moderate LOS with multipath (0 < K < 10 dB)")
            else:
                print(f"  → NLOS dominant (K < 0 dB)")
        else:
            print(f"  Channel Type: NLOS (no direct path)")
            print(f"  Dominant: {dominant_type}")

        # Individual path details
        print(f"\nPropagation Paths (strongest {link_data['num_paths_shown']}):")
        for i, path in enumerate(link_data['paths'], 1):
            los_marker = " [LOS]" if path['is_los'] else ""
            interactions = ", ".join(path['interaction_types']) if path['interaction_types'] else "direct"
            doppler = f", Doppler: {path.get('doppler_hz', 0):.1f} Hz" if path.get('doppler_hz') is not None else ""

            print(f"  Path {i}: {path['delay_ns']:.2f} ns, {path['power_db']:.1f} dB{los_marker}")
            print(f"          Interactions: {interactions}{doppler}")

In [ ]:
# Cell 4: Main Visualization Loop
async def visualization_loop() -> None:
    """
    Main loop for real-time visualization.

    Polls channel server every second for cached visualization state.
    No computation performed in notebook - instant updates.
    """
    iteration = 0
    while True:
        try:
            clear_output(wait=True)
            print(f"=== Real-Time Visualization (Iteration {iteration}) ===")
            print(f"Update interval: {UPDATE_INTERVAL_SEC}s\n")

            # Fetch cached visualization state (instant - no computation)
            viz_state = await fetch_visualization_state()

            # Display state
            display_visualization(viz_state)

            iteration += 1
            await asyncio.sleep(UPDATE_INTERVAL_SEC)

        except KeyboardInterrupt:
            print("\nVisualization stopped.")
            break
        except Exception as e:
            print(f"Error: {e}")
            await asyncio.sleep(UPDATE_INTERVAL_SEC)

# Run the loop
await visualization_loop()

In [ ]:
# Cell 5: One-Time Snapshot (Alternative to Loop)
async def render_snapshot() -> None:
    """
    Render a single snapshot of current visualization state.
    
    Use this instead of the continuous loop if you just want
    to see the current state once.
    """
    # Fetch current state
    viz_state = await fetch_visualization_state()

    # Display
    display_visualization(viz_state)

# Run snapshot
await render_snapshot()